# Embedding이란?
**텍스트 → 수치 벡터(vector)** 로 변환하는 작업  
즉, 사람의 언어를 컴퓨터가 이해할 수 있는 형식으로 바꾸는 것.

=> `LangChain`에서 관련있는 문서를 검색 해 올 수 있어야 하기 때문에,   
   문서를 수치 벡터화 시켜, 의미가 비슷한 문서를 찾을 수 있다.

### HuggingFace Embeddings

`HuggingFaceEmbeddings`는 LangChain에서 HuggingFaceHub의   
다양한 임베딩 모델을 활용할 수 있게 해주는 통합 인터페이스입니다.

**특징**
- 로컬 실행: 인터넷 연결 없이도 사용 가능
- 빠른 속도: 로컬에서 직접 실행되어 네트워크 지연 없음
- 무료 사용: API 비용 없음
- 개인정보 보호: 데이터가 외부로 전송되지 않음

**주요 매개변수**
- `model_name`: 사용할 HuggingFace 모델명
- `model_kwargs`: 모델 초기화 시 전달할 파라미터
- `encode_kwargs`: 인코딩 시 전달할 파라미터
- `cache_folder`: 모델 캐시 저장 경로


nlpai-lab/KURE-v1은 sentence-transformers를 기반으로 동작하는 모델입니다.

In [ ]:
%pip install sentence-transformers

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 1. 임베딩 객체 생성
embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 모델 다운로드드
    model_kwargs={'device': 'cpu'},  # CPU 사용 (GPU가 있다면 'cuda'도 가능)
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화 활성화
)

print(f"모델명: {embeddings.model_name}")
print("모델 로딩 완료!")


In [ ]:
# 2. 단일 쿼리 임베딩 생성
text = "LangChain은 대화형 AI 애플리케이션 개발을 위한 프레임워크입니다."
print(f"원본 텍스트: {text}")

# 임베딩 생성
query_embedding = embeddings.embed_query(text)

print(f"임베딩 차원: {len(query_embedding)}")
print(f"임베딩 벡터 (처음 5개 값): {query_embedding[:5]}")

In [ ]:
# 3. 여러 문서의 임베딩 생성
documents = [
    "LangChain은 대화형 AI 애플리케이션을 구축하기 위한 오픈소스 프레임워크입니다.",
    "HuggingFace는 자연어 처리를 위한 최고의 플랫폼 중 하나입니다.",
    "임베딩은 텍스트를 고차원 벡터로 변환하는 기술입니다.",
    "벡터 데이터베이스는 임베딩을 효율적으로 저장하고 검색할 수 있습니다.",
    "RAG는 검색 증강 생성을 의미하며, 외부 지식을 활용하여 응답을 생성합니다."
]

doc_embeddings = embeddings.embed_documents(documents)

print(f"임베딩 벡터: {query_embedding[:5]}")

In [ ]:
!pip install numpy

임베딩 유사도 계산

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 쿼리와 문서들 간의 코사인 유사도 한 번에 계산
query_text = "Langchain에 대해 알려주세요"
query_emb = embeddings.embed_query(query_text) # 쿼리 임베딩

doc_matrix = np.array(doc_embeddings)

# reshape
query_vector = np.array(query_emb).reshape(1, -1)


# 벡터화된 코사인 유사도 계산
similarities = cosine_similarity(query_vector, doc_matrix)[0]

# 출력
print(f"쿼리: {query_text}")
print("\n문서별 유사도:")
for i, (sim, doc) in enumerate(zip(similarities, documents)):
    print(f"문서 {i+1} (유사도: {sim:.4f}): {doc[:50]}...")

# 가장 유사한 문서 출력
most_similar_idx = np.argmax(similarities)
print(f"\n가장 유사한 문서: {documents[most_similar_idx]}")
